使用sklearn库

n_neighbors：默认为5，就是k-NN的k的值，选取最近的k个点。

weights：默认是uniform，参数可以是uniform、distance，也可以是用户自己定义的函数。uniform是均等的权重，就说所有的邻近点的权重都是相等的。distance是不均等的权重，距离近的点比距离远的点的影响大。用户自定义的函数，接收距离的数组，返回一组维数相同的权重。

algorithm：快速k近邻搜索算法，默认参数为auto，可以理解为算法自己决定合适的搜索算法。除此之外，用户也可以自己指定搜索算法ball_tree、kd_tree、brute方法进行搜索，brute是蛮力搜索，也就是线性扫描，当训练集很大时，计算非常耗时。kd_tree，构造kd树存储数据以便对其进行快速检索的树形数据结构，kd树也就是数据结构中的二叉树。以中值切分构造的树，每个结点是一个超矩形，在维数小于20时效率高。ball tree是为了克服kd树高纬失效而发明的，其构造过程是以质心C和半径r分割样本空间，每个节点是一个超球体。

leaf_size：默认是30，这个是构造的kd树和ball树的大小。这个值的设置会影响树构建的速度和搜索速度，同样也影响着存储树所需的内存大小。需要根据问题的性质选择最优的大小。

metric：用于距离度量，默认度量是minkowski，也就是p=2的欧氏距离(欧几里德度量)。

p：距离度量公式。在上小结，我们使用欧氏距离公式进行距离度量。除此之外，还有其他的度量方法，例如曼哈顿距离。这个参数默认为2，也就是默认使用欧式距离公式进行距离度量。也可以设置为1，使用曼哈顿距离公式进行距离度量。

metric_params：距离公式的其他关键参数，这个可以不管，使用默认的None即可。
n_jobs：并行处理设置。默认为1，临近点搜索并行工作数。如果为-1，那么CPU的所有cores都用于并行工作。


In [ ]:
# from PIL import Image
# import numpy 

# img = Image.open('/home/tzr/Documents/GitHubSYNC/PythonandMLLearning/MLLearning/KNN/trainingDataRocklab/Test1/F11-1Test.png')
# img.show()
# w,h = img.size


In [ ]:
# img.convert('L')

In [ ]:
# numpy.array(img) 

In [ ]:
# pngpath = '/home/tzr/Documents/GitHubSYNC/PythonandMLLearning/MLLearning/KNN/trainingDataRocklab/png/'
# txtpath = '/home/tzr/Documents/GitHubSYNC/PythonandMLLearning/MLLearning/KNN/trainingDataRocklab/txt/'


In [ ]:
# 图片转化
from PIL import Image
import numpy
from os import listdir
pngpath = '/home/tzr/Documents/RockLabData/KNN_1/png/' #注意单斜杠
txtpath = '/home/tzr/Documents/RockLabData/KNN_1/txt/' #注意单斜杠
def png2txt():
    '''将固定尺寸的手写体图片转换成0,1数组矩阵文件'''
    pnglist = listdir(pngpath) #listdir()可输出文件夹下所有的文件名
    for i in pnglist:
        im = Image.open(pngpath+i) #读取图片
        fh = open(txtpath+i,"a") #打开待保存的文档
        for m in range(0,500): #此为图片的像素高度，也可以用width = im.size()[1]提取
            for n in range(0,2400):#此为图片的像素宽度，也可以用width = im.size()[0]提取
                pix = im.getpixel((n,m)) #提取像素
                pixs = pix[0]+pix[1]+pix[2] 
                if pixs==255+255+255: #RGB数值相加为白色
                    fh.write("0") #黑色用1表示
                else:
                    fh.write("1") #白色用0表示
            fh.write("\n") #每一行的末尾输入换行符
        fh.close() #关闭文档

png2txt()

In [ ]:
# 重命名
import os
PATH = "/home/tzr/Documents/RockLabData/KNN_1/txt"
XML_ETX = ".xml"
IMG_ETX = ".jpg"
file_id = 0
 
def rename_file(oldname):
    global file_id
    oldname = os.path.join(PATH, oldname)
    # newname = oldname 
    # 1：方形试样
    # 2：圆形试样
    newname = "2-1%03d" % file_id
    # newname = os.path.join(PATH,newname)+'.txt'
    newname = os.path.join(PATH,newname)
    # newname = newname+"1"

    os.rename(oldname, newname)
    # print("oldname -------------> newname ", oldname,newname)
    file_id +=1
 
if __name__=="__main__":
    fileList = os.listdir(PATH)
    fileList.sort()
    for name in fileList:
        rename_file(name)

In [ ]:
# KNN分类器

In [2]:
# -*- coding: UTF-8 -*-
import numpy as np
import operator
from os import listdir
from sklearn.neighbors import KNeighborsClassifier as kNN

"""
函数说明:向量转化。

Parameters:
    filename - 文件名
Returns:
    returnVect - 返回的二进制图像的向量

Modify:
    ---
"""


def img2vector(filename):
    # 创建向量
    returnVect = np.zeros((1, 1200000))
    # 打开文件
    fr = open(filename)
    # 按行读取
    for i in range(500):
        # 读一行数据
        lineStr = fr.readline()
        # 每一行的前32个元素依次添加到returnVect中
        for j in range(2400):
            returnVect[0, 2400*i+j] = int(lineStr[j])
    # 返回转换后的向量
    return returnVect


"""
函数说明:分类测试

Parameters:
    无
Returns:
    无

Modify:
    ---
"""


def KNNclassify():
    # 测试集的Labels
    hwLabels = []
    # 返回trainingDigits目录下的文件名，输入数据位置
    trainingFileList = listdir('/home/tzr/Documents/RockLabData/KNN_1/Training6/')
    # 返回文件夹下文件的个数
    m = len(trainingFileList)
    # 初始化训练的Mat矩阵,测试集
    trainingMat = np.zeros((m, 1200000))
    # 从文件名中解析出训练集的类别
    for i in range(m):
        # 获得文件的名字
        fileNameStr = trainingFileList[i]
        # 获得分类的数字
        classNumber = int(fileNameStr.split('-')[0])
        # 将获得的类别添加到hwLabels中
        hwLabels.append(classNumber)
        # 将每一个文件的1x1024数据存储到trainingMat矩阵中
        trainingMat[i, :] = img2vector(
            '/home/tzr/Documents/RockLabData/KNN_1/Training6/%s' % (fileNameStr))
        print(i)
    # 构建kNN分类器
    neigh = kNN(n_neighbors=3, algorithm='auto')
    #拟合模型, trainingMat为训练矩阵,hwLabels为对应的标签
    neigh.fit(trainingMat, hwLabels)
    # 返回testDigits目录下的文件列表
    testFileList = listdir('/home/tzr/Documents/RockLabData/KNN_1/Test6')
    # 错误检测计数
    errorCount = 0.0
    # 测试数据的数量
    mTest = len(testFileList)
    # 从文件中解析出测试集的类别并进行分类测试
    for i in range(mTest):
        # 获得文件的名字
        fileNameStr = testFileList[i]
        # 获得分类的数字
        classNumber = int(fileNameStr.split('-')[0])
        # 获得测试集的1x1024向量,用于训练
        vectorUnderTest = img2vector(
            '/home/tzr/Documents/RockLabData/KNN_1/Test6/%s' % (fileNameStr))
        # 获得预测结果
        # classifierResult = classify0(vectorUnderTest, trainingMat, hwLabels, 3)
        classifierResult = neigh.predict(vectorUnderTest)
        if classifierResult == 1:
            res = '圆形'
        if classifierResult == 2:
            res = '方形'
        if classNumber == 1:
            rockclass = '圆形'
        if classNumber == 2:
            rockclass = '方形'
        print("分类返回结果为%s\t真实结果为%s" % (res, rockclass))
        if(classifierResult != classNumber):
            errorCount += 1.0
    print("总共错了%d个数据\n错误率为%f%%，正确率为%f%%" %
          (errorCount, errorCount/mTest * 100, (1 - errorCount/mTest )* 100))


"""
函数说明:main函数

Parameters:
    无
Returns:
    无

Modify:
    ---
"""
if __name__ == '__main__':
    KNNclassify()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
分类返回结果为方形	真实结果为方形
分类返回结果为方形	真实结果为方形
分类返回结果为圆形	真实结果为圆形
分类返回结果为方形	真实结果为方形
分类返回结果为圆形	真实结果为圆形
分类返回结果为方形	真实结果为方形
分类返回结果为圆形	真实结果为圆形
分类返回结果为圆形	真实结果为圆形
分类返回结果为方形	真实结果为方形
分类返回结果为圆形	真实结果为圆形
总共错了0个数据
错误率为0.000000%，正确率为100.000000%


In [ ]:
# KNN在两种岩石中可轻易识别